In [11]:
!wget https://raw.githubusercontent.com/meytiii/Deep-Learning/main/datasets/6-persian-topics.zip

--2024-04-27 17:01:10--  https://raw.githubusercontent.com/meytiii/Deep-Learning/main/datasets/6-persian-topics.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16473431 (16M) [application/zip]
Saving to: ‘6-persian-topics.zip.1’

6-persian-topics.zi 100%[===================>]  15.71M  --.-KB/s    in 0.09s   

2024-04-27 17:01:11 (167 MB/s) - ‘6-persian-topics.zip.1’ saved [16473431/16473431]



**Extracting the ZipFile.**

In [12]:
import zipfile
import os

In [13]:
zip_path = '/content/6-persian-topics.zip'
extracting_dir = '/content/'
os.makedirs(extracting_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracting_dir)
extracted_files = os.listdir(extracting_dir)

**Loading the required libraries.**

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

**Label each file to the according category**

In [15]:
data = []
labels = []
categories = os.listdir("/content/6-persian-topics")

for category in categories:
    files = os.listdir(os.path.join("/content/6-persian-topics", category))
    for file in files:
        with open(os.path.join("/content/6-persian-topics", category, file), "r", encoding="utf-8") as f:
            text = f.read()
            data.append(text)
            labels.append(category)

**All the texts are now in one list with each category sticked to them.**

In [16]:
categories

['بهداشت و سلامت',
 'ورزش',
 'ریاضیات',
 'دین و اعتقاد',
 'جغرافیا و مکانها',
 'فناوری و علوم کاربردی و تکنولوژی']

**Split the data**

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

**Vectorize the text**

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_sequence_length = max(len(sequence) for sequence in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

In [19]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

**Define and Train the model**

In [20]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    GRU(128),
    Dense(len(categories), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test_encoded))

Epoch 1/10
940/940 [==============================] - 79s 82ms/step - loss: 0.2831 - accuracy: 0.9077 - val_loss: 0.1283 - val_accuracy: 0.9613
Epoch 2/10
940/940 [==============================] - 33s 35ms/step - loss: 0.0581 - accuracy: 0.9807 - val_loss: 0.1114 - val_accuracy: 0.9670
Epoch 3/10
940/940 [==============================] - 29s 30ms/step - loss: 0.0213 - accuracy: 0.9939 - val_loss: 0.1110 - val_accuracy: 0.9710
Epoch 4/10
940/940 [==============================] - 28s 30ms/step - loss: 0.0090 - accuracy: 0.9972 - val_loss: 0.1070 - val_accuracy: 0.9746
Epoch 5/10
940/940 [==============================] - 25s 27ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.1109 - val_accuracy: 0.9722
Epoch 6/10
940/940 [==============================] - 26s 27ms/step - loss: 0.0062 - accuracy: 0.9982 - val_loss: 0.1112 - val_accuracy: 0.9749
Epoch 7/10
940/940 [==============================] - 26s 27ms/step - loss: 0.0050 - accuracy: 0.9986 - val_loss: 0.1332 - val_accuracy:

**Evaluate the model**

Fit and transform labels in testing set

In [21]:
y_test_encoded = label_encoder.fit_transform(y_test)

Evaluate the model with the encoded labels

In [22]:
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)

235/235 [==============================] - 3s 12ms/step - loss: 0.1201 - accuracy: 0.9767
Test Accuracy: 0.976713240146637


**Let's see the accuracy :**

In [26]:
accuracy

0.976713240146637

**An example of correct prediction :**

In [29]:
random_example = "قضیه شارکوفسکی در ریاضیات سامانه پویا در سیستم گسسته مطرح شده است. الکساندر شارکوفسکی این قضیه را در سال ۱۹۶۴ ارائه و اثبات کرد."
random_example_sequence = tokenizer.texts_to_sequences([random_example])
random_example_padded = pad_sequences(random_example_sequence, maxlen=max_sequence_length)
predictions = model.predict(random_example_padded)
predicted_topic_index = np.argmax(predictions)
predicted_topic = categories[predicted_topic_index]

print(random_example)
print("Predicted Topic:", predicted_topic)

1/1 [==============================] - 0s 23ms/step
قضیه شارکوفسکی در ریاضیات سامانه پویا در سیستم گسسته مطرح شده است. الکساندر شارکوفسکی این قضیه را در سال ۱۹۶۴ ارائه و اثبات کرد.
Predicted Topic: دین و اعتقاد


**An example of incorrect prediction :**

In [30]:
random_example = "تک‌سرشت‌باوری (به انگلیسی: Monophysitism) موقعیتی در مسیح‌شناسی است که در آن پس از یگانه شدن سرشت خدا و انسان، عیسی مسیح به عنوان تجسم پسر یا کلمه خدا، تنها یک ذات یا طبیعت دارد که الهی است. تک‌سرشت‌باوری با دوسرشت‌باوری در تضاد است که معتقد است مسیح دارای دو سرشت، یکی الهی و یکی انسانی است."
random_example_sequence = tokenizer.texts_to_sequences([random_example])
random_example_padded = pad_sequences(random_example_sequence, maxlen=max_sequence_length)
predictions = model.predict(random_example_padded)
predicted_topic_index = np.argmax(predictions)
predicted_topic = categories[predicted_topic_index]

print(random_example)
print("Predicted Topic:", predicted_topic)

1/1 [==============================] - 0s 25ms/step
تک‌سرشت‌باوری (به انگلیسی: Monophysitism) موقعیتی در مسیح‌شناسی است که در آن پس از یگانه شدن سرشت خدا و انسان، عیسی مسیح به عنوان تجسم پسر یا کلمه خدا، تنها یک ذات یا طبیعت دارد که الهی است. تک‌سرشت‌باوری با دوسرشت‌باوری در تضاد است که معتقد است مسیح دارای دو سرشت، یکی الهی و یکی انسانی است.
Predicted Topic: ریاضیات
